In [1]:
import spacy
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
data = pd.read_json('../data/restaurant.json')
data.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
109,lDJIaF4eYRF4F7g6Zb9euw,lb0QUR5bc4O-Am4hNq9ZGg,r5PLDU-4mSbde5XekTXSCA,4,2,0,0,I used to work food service and my manager at ...,2013-01-27 17:54:54
1013,vvIzf3pr8lTqE_AOsxmgaA,MAmijW4ooUzujkufYYLMeQ,r5PLDU-4mSbde5XekTXSCA,4,0,0,0,We have been trying Eggplant sandwiches all ov...,2015-04-15 04:50:56
1204,UF-JqzMczZ8vvp_4tPK3bQ,slfi6gf_qEYTXy90Sw93sg,r5PLDU-4mSbde5XekTXSCA,5,1,0,0,Amazing Steak and Cheese... Better than any Ph...,2011-03-20 00:57:45
1251,geUJGrKhXynxDC2uvERsLw,N_-UepOzAsuDQwOUtfRFGw,r5PLDU-4mSbde5XekTXSCA,1,0,0,0,Although I have been going to DeFalco's for ye...,2018-07-17 01:48:23
1354,aPctXPeZW3kDq36TRm-CqA,139hD7gkZVzSvSzDPwhNNw,r5PLDU-4mSbde5XekTXSCA,2,0,0,0,"Highs: Ambience, value, pizza and deserts. Thi...",2018-01-21 10:52:58


In [3]:
menu = ["Cheese Steak", "Cheesesteak", "Steak and Cheese", "Italian Combo", "Tiramisu", "Cannoli",
        "Chicken Salad", "Chicken Spinach Salad", "Meatball", "Pizza", "Pizzas", "Spaghetti",
        "Bruchetta", "Eggplant", "Italian Beef", "Purista", "Pasta", "Calzones",  "Calzone",
        "Italian Sausage", "Chicken Cutlet", "Chicken Parm", "Chicken Parmesan", "Gnocchi",
        "Chicken Pesto", "Turkey Sandwich", "Turkey Breast", "Ziti", "Portobello", "Reuben",
        "Mozzarella Caprese",  "Corned Beef", "Garlic Bread", "Pastrami", "Roast Beef",
        "Tuna Salad", "Lasagna", "Artichoke Salad", "Fettuccini Alfredo", "Chicken Parmigiana",
        "Grilled Veggie", "Grilled Veggies", "Grilled Vegetable", "Mac and Cheese", "Macaroni",  
         "Prosciutto", "Salami"]

In [4]:
nlp = spacy.blank('en')
matcher = spacy.matcher.PhraseMatcher(nlp.vocab, attr='LOWER')
docs_menu = [nlp(item) for item in menu]
matcher.add('menu', docs_menu)
item_ratings = defaultdict(list)

In [5]:
for idx, review in data.iterrows():
    doc_review = nlp(review['text'])
    matches = matcher(doc_review)
    found_items = set([doc_review[match[1]:match[2]].text.lower() for match in matches])
    for item in found_items:
        item_ratings[item].append(review.stars)

In [6]:
mean_ratings = {k:np.mean(v) for k, v in item_ratings.items()}

lowest_rating = np.inf
for k, v in mean_ratings.items():
    if v < lowest_rating:
        worst_item = k
        lowest_rating = v

In [7]:
print(worst_item)
print(mean_ratings[worst_item])

chicken cutlet
3.4


In [8]:
counts = {item: len(ratings) for item, ratings in item_ratings.items()}

item_counts = sorted(counts, key=counts.get, reverse=True)
for item in item_counts:
    print(f"{item:>25}{counts[item]:>5}")

                    pizza  265
                    pasta  206
                 meatball  128
              cheesesteak   97
             cheese steak   76
                  cannoli   72
                  calzone   72
                 eggplant   69
                  purista   63
                  lasagna   59
          italian sausage   53
               prosciutto   50
             chicken parm   50
             garlic bread   39
                  gnocchi   37
                spaghetti   36
                 calzones   35
                   pizzas   32
                   salami   28
            chicken pesto   27
             italian beef   25
                 tiramisu   21
            italian combo   21
                     ziti   21
         chicken parmesan   19
       chicken parmigiana   17
               portobello   14
           mac and cheese   11
           chicken cutlet   10
         steak and cheese    9
                 pastrami    9
               roast beef    7
       f

In [9]:
sorted_ratings = sorted(mean_ratings, key=mean_ratings.get)

print("Worst rated menu items:")
for item in sorted_ratings[:10]:
    print(f"{item:20} Ave rating: {mean_ratings[item]:.2f} \tcount: {counts[item]}")
    
print("\n\nBest rated menu items:")
for item in sorted_ratings[-10:]:
    print(f"{item:20} Ave rating: {mean_ratings[item]:.2f} \tcount: {counts[item]}")

Worst rated menu items:
chicken cutlet       Ave rating: 3.40 	count: 10
turkey sandwich      Ave rating: 3.80 	count: 5
spaghetti            Ave rating: 3.89 	count: 36
italian beef         Ave rating: 3.92 	count: 25
tuna salad           Ave rating: 4.00 	count: 5
macaroni             Ave rating: 4.00 	count: 5
italian combo        Ave rating: 4.05 	count: 21
garlic bread         Ave rating: 4.13 	count: 39
roast beef           Ave rating: 4.14 	count: 7
eggplant             Ave rating: 4.16 	count: 69


Best rated menu items:
chicken pesto        Ave rating: 4.56 	count: 27
chicken salad        Ave rating: 4.60 	count: 5
purista              Ave rating: 4.67 	count: 63
prosciutto           Ave rating: 4.68 	count: 50
reuben               Ave rating: 4.75 	count: 4
steak and cheese     Ave rating: 4.89 	count: 9
artichoke salad      Ave rating: 5.00 	count: 5
fettuccini alfredo   Ave rating: 5.00 	count: 6
turkey breast        Ave rating: 5.00 	count: 1
corned beef          Ave ratin